## Методы решения бигармонических уравнений
#### Метод отражения Хаусхолдера

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sympy
from sympy import diff
import sys

In [27]:
#Variances:
L = 1 #Border
K = 5
h = L/(2*K) # width of cell
N = 4 #power of the polynoms
k = [5,10,20,40,80] #Number of cells

c0 = 0.4
eps = 10^(-12)

c_start = c0*np.ones(K*(N+1))

plt.figure(figsize=(16,50))


<Figure size 1152x3600 with 0 Axes>

<Figure size 1152x3600 with 0 Axes>

In [3]:
def sign(num):
    return -1 if num < 0 else 1

In [4]:
def coll_dot():
    colloc_mas = np.zeros([N+1]) 
    for m in range (N+1):
        colloc_mas[m] = np.cos((2*(m+1)-1)*np.pi/(2*(N+1)))
    colloc_mas = np.flipud(colloc_mas)
    return colloc_mas

In [5]:
#Chebyshev polynoms and their diff
def Chebishev(point, n):
    x = sympy.Symbol('x')
    v  = sympy.chebyshevt(n, x)
    return v.subs(x,point)

def Chebishev_dx(point, n):
    x = sympy.Symbol('x')
    v  = sympy.chebyshevt(n, x)
    v = diff(v,x)
    return v.subs(x,point)

def Chebishev_d2x(point, n):
    x = sympy.Symbol('x')
    v  = sympy.chebyshevt(n, x)
    v = diff(v,x,2)
    return v.subs(x,point)   

def Chebishev_d3x(point, n):
    x = sympy.Symbol('x')
    v  = sympy.chebyshevt(n, x)
    v = diff(v,x,3)
    return v.subs(x,point) 

def Chebishev_d4x(point, n):
    x = sympy.Symbol('x')
    v  = sympy.chebyshevt(n, x)
    v = diff(v,x,4)
    return v.subs(x,point) 

#D4 of the solution
def Func(x_point):
    x = sympy.Symbol('x')
    f= x**2*(1-x)**2*sympy.exp(x)
    f = diff(f,x,4)
    #sympy.pprint(dx)
    return f.subs(x,x_point) 

#F of the solution
def Acc_Func(x_point):
    x = sympy.Symbol('x')
    f  = x**2*(1-x)**2*sympy.exp(x)
    return f.subs(x,x_point) 


In [6]:
#Left c_vector
def b_left(m, dots, c_old):
    b = np.zeros(N+5)
    
    for i in range (m):
        for j in range(N+1):
            b[j] = h**4*Func(h*dots[j-2] + h*(2*i+1))
            
    for j in range (N+1):
        b[N+1] = b[N+3] + c_old[j]*(Chebishev(1,j) + Chebishev_dx(1,j))
        b[N+2] = b[N+4] + c_old[j]*(Chebishev_d2x(1,j) + Chebishev_d3x(1,j))
    return b

In [7]:
#Coll c_vector
def b_mid(m, dots, c_old):
    b = np.zeros(N+5)
    
    for j in range (N+1):
        b[N+1] = b[N+1] + c_old[j]*(-(Chebishev(-1,j) + Chebishev_dx(-1,j)))
        b[N+2] = b[N+2] + c_old[j]*(-(Chebishev_d2x(-1,j) + Chebishev_d3x(-1,j)))
    for i in range (m):
        for j in range(N+1):
            b[j] = h**4*Func(h*dots[j-2] + h*(2*i+1))
            
    for j in range (N+1):
        b[N+3] = b[N+3] + c_old[j]*(Chebishev(1,j) + Chebishev_dx(1,j))
        b[N+4] = b[N+4] + c_old[j]*(Chebishev_d2x(1,j) + Chebishev_d3x(1,j))
    return b

In [8]:
#Right c_vector
def b_right(m_from_k, dots, c_old):
    b = np.zeros(N+5)
    
    for j in range (N+1):
        b[N+1] = b[N+1] + c_old[j]*(-(Chebishev(-1,j) + Chebishev_dx(-1,j)))
        b[N+2] = b[N+2] + c_old[j]*(-(Chebishev_d2x(-1,j) + Chebishev_d3x(-1,j)))
    
    for i in range (m_from_k):
        for j in range(N+1):
            b[j] = h**4*Func(h*dots[j-2] + h*(2*i+1))
    return b

In [9]:
def First_block(A, dots):
    for j in range (N+1):
        for i in range (N+1):
            A[j][i] = Chebishev_d4x(dots[j - 2],i)
    for j in range (N+1):
        A[N+1][j] = Chebishev(1,j) + Chebishev_dx(1,j)
        A[N+2][j] = Chebishev_d2x(1,j) + Chebishev_d3x(1,j)
        A[N+3][j] = Chebishev(-1,j)
        A[N+4][j] = Chebishev_dx(-1,j)
    return A

In [10]:
def Coll_block(A, dots):
    for j in range (N+1):
        for i in range (N+1):
            A[j][i] = Chebishev_d4x(dots[j - 2],i)
    for j in range (N+1):
        A[N+1][j] = -(Chebishev(-1,j) + Chebishev_dx(-1,j))
        A[N+2][j] = -(Chebishev_d2x(-1,j) + Chebishev_d3x(-1,j))
        A[N+3][j] = Chebishev(1,j) + Chebishev_dx(1,j)
        A[N+4][j] = Chebishev_d2x(1,j) + Chebishev_d3x(1,j)
        return A

In [11]:
def Last_block(A, dots):
    for j in range (N+1):
        for i in range (N+1):
            A[j][i] = Chebishev_d4x(dots[j - 2],i)
    for j in range (N+1):
        A[N+1][j] = -(Chebishev(-1,j) + Chebishev_dx(-1,j))
        A[N+2][j] = -(Chebishev_d2x(-1,j) + Chebishev_d3x(-1,j))
        A[N+3][j] = Chebishev(1,j)
        A[N+4][j] = Chebishev_dx(1,j)
   
    return A

In [12]:
dd = coll_dot()
AAA = np.zeros((N+5,N+1))
for j in range (2,N+3):
    for i in range (N+1):
        AAA[j][i] = Chebishev_d4x(dd[j-2],i)
print(AAA)
print(dd)
print(AAA[2,4])
print(AAA.shape)
print(AAA[0:3,3:4])

[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]]
[-9.51056516e-01 -5.87785252e-01  6.12323400e-17  5.87785252e-01
  9.51056516e-01]
192.0
(9, 5)
[[0.]
 [0.]
 [0.]]


In [13]:
def BackGauss (Aq, Yq):   
#     x = np.zeros(N+1)
#     for i in range(0, N+1):
#         sum = 0
#         for j in range(N-i+1,N):
#             sum = sum + Aq[N-i][j]*x[j]

#         x[N-i] = (Yq[N-i] - sum) / Aq[N-i][N-i]
    n = Aq.shape[0]
    x = np.zeros(n)
    for i in range(n):
        for j in range(n):
            Yq[n - i-1] = Yq[n - i-1] - x[j]*Aq[n - i-1][n - j-1]
        x[i] = Yq[n - i-1] / Aq[n - i-1][n - i-1]
    x = np.flipud(x)      
    return x

In [25]:
def Householder(Mtx, b_vect):
    m,k = Mtx.shape
    x = np.zeros(N+1)
    Ac = Mtx.copy()
    bc = b_vect.copy()
    for i in range (k):
        if any(Ac[i:k, i] != 0 ):
            u = np.copy(Ac[i:m, i])
            u[0] += np.linalg.norm(u)*sign(u[0])
            u.shape = (m - i , 1)
            bc.shape = (m , 1)
            Ac[i:m,i:m] -= 2*np.dot(u, np.dot(u.T, np.copy(Ac[i:m, 
                                            i:m])))/np.dot(u.T, u)
            bc[i:m] -= 2*np.dot(u, np.dot(u.T, np.copy(bc[i:m])))/np.dot(u.T, u)
            
#     for i in range(0,k):
#         for j in range(0,m):
#             if abs(Ac[j][i]) < 10**(-10):
#                 Ac[j][i] = 0

    Aq = Ac[0:k, 0:k]
    Yq = bc[0:k]
    print(Aq)
    x = BackGauss(Aq, Yq)
    
    return x 

In [15]:
#Iterations on subcells
def Solving(K):
    i = 1
    A = np.zeros([(N+5),(N+1)])
    colloc_mas = coll_dot()
    err = 1
    c_prev = c_start
    c_calculated = np.zeros(K*(N+1))
    
    while err > eps:
        
        while i < K+1:
            if i == 1:
                A = np.zeros([(N+5),(N+1)])
                b = np.zeros([N+5, 1])
                A = First_block(A, colloc_mas)
                print(A)
                b = b_left(i, colloc_mas, c_prev)
                print(b_left)
                c_calculated = Householder(A, b)
                err = np.amax(c_prev - c_calculated)
                c_prev = c_calculated
                i += 1
            elif i == K:
                A = np.zeros([(N+5),(N+1)])
                b = np.zeros([N+5, 1])
                A = Last_block(A, colloc_mas)
                print(A)
                b = b_right(i, colloc_mas, c_prev)
                print(b)
                c_calculated = Householder(A, b)
                err = np.amax(c_prev - c_calculated)
                c_prev = c_calculated
                i = 1
            else:
                A = np.zeros([(N+5),(N+1)])
                b = np.zeros([N+5, 1])
                A = Coll_block(A, colloc_mas)
                print(A)
                b = b_mid(i, colloc_mas, c_prev)
                c_calculated = Householder(A, b)
                err = np.amax(c_prev - c_calculated)
                c_prev = c_calculated
                i += 1
                
                
    return c_calculated           

In [16]:
def E_r(v, w):
    d= [0]*len(w)
    for i in range (len(w)):
        d[i] = np.abs(w[i] - v[i])
    return max(d)/max(w)

def E_a(w, v):
    d= [0]*len(w)
    for i in range (len(w)):
        d[i] = np.abs(w[i] - v[i])
    return max(d)

In [26]:
Solving(3)

[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -1.   1.  -1.   1.]
 [  0.   1.  -4.   9. -16.]]
<function b_left at 0x000002078BC5F790>
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -3.33757397e+02]
 [ 1.00000000e+00  2.00000000e+00  5.00000000e+00  1.00000000e+01
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  4.00000000e+00  4.80000000e+01
   0.00000000e+00]
 [ 1.00000000e+00 -1.00000000e+00  1.00000000e+00 -1.00000000e+00
  -2.22044605e-16]
 [ 0.00000000e+00  1.00000000e+00 -4.00000000e+00  9.00000000e+00
   3.55271368e-15]]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [ -1.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  1.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]]
[[   0.    0.    0.    0. -1

C:\Users\Slava\AppData\Local\Temp/ipykernel_16828/2942579673.py:14: RuntimeWarning: divide by zero encountered in true_divide
  x[i] = Yq[n - i-1] / Aq[n - i-1][n - i-1]


[0.0041476  0.00498143 0.00149102 0.00200523 0.002978          nan
        nan 0.         0.        ]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.23360695e+02]
 [-1.00000000e+00  0.00000000e+00  3.00000000e+00 -8.00000000e+00
   3.55271368e-15]
 [ 0.00000000e+00  0.00000000e+00 -4.00000000e+00  0.00000000e+00
   1.42108547e-14]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.11022302e-16]
 [ 0.00000000e+00  1.00000000e+00  4.00000000e+00  9.00000000e+00
   1.77635684e-15]]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -1.   1.  -1.   1.]
 [  0.   1.  -4.   9. -16.]]
<function b_left at 0x000002078BC5F790>
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -3.33757397e+02]
 [ 1.00000000e+00  2.00000000e+00  5.00000000e+00  1.00000000e+01
   0.00000000e+00]
 [ 0.0000

[0.0041476  0.00498143 0.00149102 0.00200523 0.002978          nan
        nan 0.         0.        ]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.23360695e+02]
 [-1.00000000e+00  0.00000000e+00  3.00000000e+00 -8.00000000e+00
   3.55271368e-15]
 [ 0.00000000e+00  0.00000000e+00 -4.00000000e+00  0.00000000e+00
   1.42108547e-14]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.11022302e-16]
 [ 0.00000000e+00  1.00000000e+00  4.00000000e+00  9.00000000e+00
   1.77635684e-15]]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -1.   1.  -1.   1.]
 [  0.   1.  -4.   9. -16.]]
<function b_left at 0x000002078BC5F790>
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -3.33757397e+02]
 [ 1.00000000e+00  2.00000000e+00  5.00000000e+00  1.00000000e+01
   0.00000000e+00]
 [ 0.0000

[0.0041476  0.00498143 0.00149102 0.00200523 0.002978          nan
        nan 0.         0.        ]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.23360695e+02]
 [-1.00000000e+00  0.00000000e+00  3.00000000e+00 -8.00000000e+00
   3.55271368e-15]
 [ 0.00000000e+00  0.00000000e+00 -4.00000000e+00  0.00000000e+00
   1.42108547e-14]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.11022302e-16]
 [ 0.00000000e+00  1.00000000e+00  4.00000000e+00  9.00000000e+00
   1.77635684e-15]]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -1.   1.  -1.   1.]
 [  0.   1.  -4.   9. -16.]]
<function b_left at 0x000002078BC5F790>
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -3.33757397e+02]
 [ 1.00000000e+00  2.00000000e+00  5.00000000e+00  1.00000000e+01
   0.00000000e+00]
 [ 0.0000

...

[0.0041476  0.00498143 0.00149102 0.00200523 0.002978          nan
        nan 0.         0.        ]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.23360695e+02]
 [-1.00000000e+00  0.00000000e+00  3.00000000e+00 -8.00000000e+00
   3.55271368e-15]
 [ 0.00000000e+00  0.00000000e+00 -4.00000000e+00  0.00000000e+00
   1.42108547e-14]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.11022302e-16]
 [ 0.00000000e+00  1.00000000e+00  4.00000000e+00  9.00000000e+00
   1.77635684e-15]]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -1.   1.  -1.   1.]
 [  0.   1.  -4.   9. -16.]]
<function b_left at 0x000002078BC5F790>
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -3.33757397e+02]
 [ 1.00000000e+00  2.00000000e+00  5.00000000e+00  1.00000000e+01
   0.00000000e+00]
 [ 0.0000

<function b_left at 0x000002078BC5F790>
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -3.33757397e+02]
 [ 1.00000000e+00  2.00000000e+00  5.00000000e+00  1.00000000e+01
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  4.00000000e+00  4.80000000e+01
   0.00000000e+00]
 [ 1.00000000e+00 -1.00000000e+00  1.00000000e+00 -1.00000000e+00
  -2.22044605e-16]
 [ 0.00000000e+00  1.00000000e+00 -4.00000000e+00  9.00000000e+00
   3.55271368e-15]]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [ -1.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  1.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]]
[[   0.    0.    0.    0. -192.]
 [  -1.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]
 [   1.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [ 

[0.0041476  0.00498143 0.00149102 0.00200523 0.002978          nan
        nan 0.         0.        ]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.23360695e+02]
 [-1.00000000e+00  0.00000000e+00  3.00000000e+00 -8.00000000e+00
   3.55271368e-15]
 [ 0.00000000e+00  0.00000000e+00 -4.00000000e+00  0.00000000e+00
   1.42108547e-14]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.11022302e-16]
 [ 0.00000000e+00  1.00000000e+00  4.00000000e+00  9.00000000e+00
   1.77635684e-15]]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -1.   1.  -1.   1.]
 [  0.   1.  -4.   9. -16.]]
<function b_left at 0x000002078BC5F790>
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -3.33757397e+02]
 [ 1.00000000e+00  2.00000000e+00  5.00000000e+00  1.00000000e+01
   0.00000000e+00]
 [ 0.0000

[[   0.    0.    0.    0. -192.]
 [  -1.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]
 [   1.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [ -1.   0.   3.  -8.  15.]
 [  0.   0.  -4.   0. 112.]
 [  1.   1.   1.   1.   1.]
 [  0.   1.   4.   9.  16.]]
[0.0041476  0.00498143 0.00149102 0.00200523 0.002978          nan
        nan 0.         0.        ]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.23360695e+02]
 [-1.00000000e+00  0.00000000e+00  3.00000000e+00 -8.00000000e+00
   3.55271368e-15]
 [ 0.00000000e+00  0.00000000e+00 -4.00000000e+00  0.00000000e+00
   1.42108547e-14]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.11022302e-16]
 [ 0.00000000e+00  1.00000000e+00  4.00000000e+00  9.00000000e+00
   1.77635684e-15]]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192

KeyboardInterrupt: 

In [ ]:
def Solution(N, K, points):
    C = np.full((K*(N+5)),c0)
    CP = np.zeros(K*(N+5))
    s = 0
    n = 0
    c1 = C[0:N+1]
    c2 = np.zeros((N+1))
    print(c2)

    a = 1
    
    while a==1:
        
            
        A = Mtx(n, N, K, points,c1)
        B = b_vector(N,K,n,c1,points)
        #c2 = Givens(A,B)
        
        c2 = Coeff(A,B)
        
        c1 = C[n+N+1:n + 2*(N+1)]
        print(c2)
        CP[n*(N+1):(n+1)*(N+1)] = c2
        n = n + 1
        
        #for i in range (K*(N+5)):
            #if max(CP[i] - C[i]) < eps:
                #break
                
        #for i in range (K*(N+5)):
        if max(CP - C) < eps:
                break
                
        if n > K:
            s = s + 1
            n = 0
            for l in range ((K*(N+5))):
                C[l] = CP[l]        
                
    B_Fin = A.CP
                
    return s, n, CP, B_Fin
        

In [ ]:
print("Размер сетки, К | ||Ea|| | Порядок сходимости | ||Er|| | Порядок сходимости | |μ(A)")
print("--------------------------------------------------------------------------------")
plt.figure(figsize=(16,50))
for i in range (len(k)):
    K = k[i]
    h = L/(2*K)
    colloc_mas = coll_init(N)
    X_Int = np.zeros(K*(N+1))
    for j in range (K):
        X_Int[0 + j*(N+1):(j+1)*(N+1)] = colloc_mas
        
        
    s, n, C_Fin, B_Fin = Solution(N,K,coll_dot)
    
    X = np.arange(0,1,100)
    B_Acc = Acc_Func(X)
    plt.subplot(5,1,i+1)
    plt.title("K = %d"%(K))
    plt.plot(x, y_sol, 'b', x, y_ac, 'g--')   
    
    if i == 0:
        ER_p = E_r(np.copy(y_sol),np.copy(y_ac))
        EA_p = E_a(np.copy(y_sol),np.copy(y_ac))
        print("%d | %e | %e | %e | %e | %f |"%(K_list[i], EA_p, 0 ,ER_p ,0 ,mu))
        print("--------------------------------------------------------------------------------")
    else:
        ER_n = E_r(np.copy(y_sol),np.copy(y_ac))
        EA_n = E_a(np.copy(y_sol),np.copy(y_ac))        
        print("%d | %e | %e | %e | %e | %f |"%(K_list[i], E_a(y_sol,y_ac), np.log2(EA_p/EA_n) ,
                                               E_r(y_sol,y_ac), np.log2(ER_p/ER_n),mu))
        ER_p = E_r(np.copy(y_sol),np.copy(y_ac))
        EA_p = E_a(np.copy(y_sol),np.copy(y_ac))
        print("--------------------------------------------------------------------------------")
plt.show()


In [ ]:
d = {"Ea":np.zeros(5),"OoC":np.zeros(5),"Er":np.zeros(5),"OoC":np.zeros(5),"N_iter":np.zeros(5),"mu(Ab)":np.zeros(5),"mu(Ai)":np.zeros(5)}
df = pd.DataFrame(d, index=['5', '10', '20','40','80'])

df.iat[N-1,0] = np.linalg.cond(np.dot(list[0].transpose(), list[0]))
df.iat[N-1,1] = SME(Y_int)
    
    
df.iat[N-1,2] = lg.cond(list[0])
df.iat[N-1,3] = SME(Y_Giv)

In [ ]:
def Approx_C(c, n, N):
    bx = 0
    for t in range (N+1):
        x = sympy.Symbol('x')
        bx  = bx + c[t]*(sympy.chebyshevt(t, x).subs(x,n))
    #a =  bx.subs(x,n)
    return bx
    
def Approx_diff(c, n, N, m):
    bx = 0
    for t in range (N+1):
        x = sympy.Symbol('x')
        bx  = bx + c[t]*((diff(sympy.chebyshevt(t, x),x,m).subs(x,n)))
    #bxc = diff(bx,x,m)
    #a =  bxc.subs(x,n)                    
    return bx

In [ ]:
def Mtx(n,N,K, colloc_mas,c):
    
    A = np.zeros([(N+5),(N+1)])
    
    for i in range (N+1):
        for j in range (N+1):
            A[i][j] = C_d4x(colloc_mas[i],j)
    
    if n == 0:
        for j in range (N+1):
            A[N+1][j] = Cheb(1,j) + C_dx(1,j)
            A[N+2][j] = C_d2x(1,j) + C_d3x(1,j)
            A[N+3][j] = Cheb(-1,j)
            A[N+4][j] = C_dx(-1,j)
            
    elif n == K-1:
        for j in range (N+1):
            A[N+1][j] = Cheb(-1,j) - C_dx(-1,j)
            A[N+2][j] = C_d2x(-1,j) - C_d3x(-1,j)
            A[N+3][j] = Cheb(1,j)
            A[N+4][j] = C_dx(1,j)
            
    else:
        for j in range (N+1): 
            A[N+1][j] = Cheb(1,j) + C_dx(1,j)
            A[N+2][j] = C_d2x(1,j) + C_d3x(1,j)
            A[N+3][j] = Cheb(-1,j) - C_dx(-1,j)
            A[N+4][j] = C_d2x(-1,j) - C_d3x(-1,j)
   
    #A = np.fliplr(A)
    #A = np.flipud(A)
    print(A)
    return A

In [ ]:
C = np.full((2*(N+5)),c0)
coll_mas1 = coll_init(N)
Mtx(0,N,2,coll_mas1,C)

In [ ]:
def Givens(A, Y):
    n = A.shape[1]
    M = Y.shape[0]
    print(M)
    
    AC = A.copy()
    YC = Y.copy()
     
    # Givens
    for i in range(0,n):
        for j in range(i + 1 , n+4):
            c = AC[i,i] / (AC[i,i]**2 + AC[j,i]**2) ** .5
            s = AC[j,i] / ((AC[i,i])**2 + (AC[j,i])**2) ** .5
            if ((AC[i,i])**2 + (AC[j,i])**2) == 0:
                c = 1
                s = 0
            print(c)
            tmp1 = AC[i,:] * c + AC[j,:] * s
            tmp2 = AC[i,:] * -s + AC[j,:] * c
            AC[i, :] = tmp1
            AC[j, :] = tmp2
            tmp3 = YC[i] * c + YC[j] * s
            tmp4 = YC[i] * (-s) + YC[j] * c
            YC[i] = tmp3
            YC[j] = tmp4
            
    
    # Gauss method_back
    
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        sum = 0
        for j in range(i+1, n):
            sum = sum + AC[i,j]*x[j]

        x[i] = (YC[i] - sum) / AC[i,i]
    return x


In [ ]:
def Coeff (A, Y):
    At = np.transpose(A)
    AH = np.dot(At,A)
    YH = np.dot(At,Y)
    AH_inv = lg.inv(AH)
    C = np.dot(AH_inv,YH)
    return C

def NU (N, C):    
    approx = np.zeros(M)
    for i in range(0,M):
        for n in range(0,N):
            approx[i] = approx[i] + C[n]*(data[i][0]**n)
    
    return approx

In [ ]:
s=0
i = 1
if i < 11:
    s += i 
    print (i)
    i += 1
    if i == 10:
        i = 1
    if s > 60:
        i = 50